In [1]:
import os
import cv2
import pickle
import random
import Augmentor
import numpy as np
from bing_image_downloader import downloader  # !pip install bing-image-downloader

In [3]:
def get_bing_images(search_title:str, limit:int) -> None:
    downloader.download(search_title,
                        output_dir='images',
                        limit=limit, 
                        timeout=60)

In [4]:
def augment_images(path:str) -> None:
    p = Augmentor.Pipeline(path)  # have source_directory, output_directory parameters.
    p.zoom(probability=0.3, min_factor=0.8, max_factor=1.5)
    p.flip_top_bottom(probability=0.4)
    p.flip_left_right(probability=0.4)
    p.rotate90(probability=0.75)
    p.skew(probability=0.2)
    p.shear(probability=0.2, max_shear_left=2, max_shear_right=2)
    p.rotate(probability=0.2, max_left_rotation=2, max_right_rotation=2)
    p.random_brightness(probability=0.3, min_factor=0.3, max_factor=1.2)
    p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=8)
    p.sample(200)

In [5]:
def rename_images(folder_path: str, prefix: str) -> None:
    files = os.listdir(folder_path)

    image_files = [file for file in files if file.endswith(('.jpg', '.JPG', 
                                                            '.JPEG', 'jpeg', 
                                                            '.png', '.PNG'))]
    image_files.sort()

    for i, file in enumerate(image_files):
        file_extension = os.path.splitext(file)[1].lower()

        new_name = f"{prefix}_{i+1}{file_extension}"

        old_path = os.path.join(folder_path, file)
        new_path = os.path.join(folder_path, new_name)

        os.rename(old_path, new_path)
        print(f"Renamed {file} to {new_name}")

In [13]:
CWD = os.getcwd()
DIR = str(CWD) + '/images/'
CATEGORIES = ['loaf', 'cat']

loaf_cats_folder_path = DIR + CATEGORIES[0]
cats_folder_path = DIR + CATEGORIES[1]

In [ ]:
get_bing_images(search_title='loaf cats', limit=100)
get_bing_images(search_title='cats', limit=100)

for category in CATEGORIES:
    #print(DIRECTORY + category)
    try: 
        augment_images(DIR + category)
    except Exception as e: print(e)

os.rename('/Users/egemeneroglu/Desktop/Files/Projects/loaf_cf/images/loaf cats', loaf_cats_folder_path)
os.rename('/Users/egemeneroglu/Desktop/Files/Projects/loaf_cf/images/cats', cats_folder_path)

rename_images(loaf_cats_folder_path, 'loaf_cat')
rename_images(cats_folder_path, 'cat')

In [45]:
def delete_ds_store_files(folder_path:str) -> None:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)

In [46]:
IMAGE_SIZE = 100

data = [] 

for category in CATEGORIES:
    folder = os.path.join(DIR, category)
    label = CATEGORIES.index(category)
    delete_ds_store_files(folder_path=folder)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)

        try:
            img_arr = cv2.resize(img_arr, (IMAGE_SIZE, IMAGE_SIZE))
            
        except Exception as e: 
            print('Error:', e)
            print('Image Path:', img_path)

        data.append([img_arr, label])
        
print(f'Total Data: {len(data)} images.')


Total Data: 600 images.


In [47]:
X,y = [], []
random.shuffle(data)
for features, labels in data:
    X.append(features)
    y.append(labels)

X,y = np.array(X), np.array(y)

pickle.dump(X, open('X.pkl', 'wb'))
pickle.dump(y, open('y.pkl', 'wb'))